# Step 1

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from vivarium import InteractiveContext, Artifact
from pathlib import Path
#import itertools
import pandas as pd, numpy as np

In [2]:
pip list | grep vivarium

vivarium                  2.1.1
vivarium-nih-us-cvd       0.1.dev374+gfbc7031
vivarium-public-health    2.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip freeze | grep vivarium

vivarium==2.1.1
vivarium-nih-us-cvd @ git+https://github.com/ihmeuw/vivarium_nih_us_cvd.git@fbc7031bf98e5bf3ad78923c504c14e55ffdc767
vivarium-public-health==2.1.0
Note: you may need to restart the kernel to use updated packages.


# Step 2

In [4]:
import vivarium_nih_us_cvd
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_nih_us_cvd.__file__.replace('__init__.py', 'model_specifications/nih_us_cvd.yaml'))

from vivarium.framework.configuration import build_model_specification

custom_model_specification = build_model_specification(path)

# Customizations: will be specific to your model and use case!
# Delete observer configuration
del custom_model_specification.configuration.observers
# Change population size
custom_model_specification.configuration.population.population_size = 50_000

# Delete the "metrics" component (an observer)
del custom_model_specification.components.vivarium_public_health.metrics

## Filter out components with 'Observer' or 'Stratifier' in the name
custom_model_specification.components.vivarium_nih_us_cvd.components = [
    c for c in custom_model_specification.components.vivarium_nih_us_cvd.components
    if 'Observer' not in c and 'Stratifier' not in c
]

## This section is specific to the CVD sim and not needed for other interactive sims! 
custom_model_specification.configuration.update({'input_data': {'input_draw_number': 0, 'artifact_path': '/mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/artifacts/51-locations/v4-20231218/alabama.hdf'}})

In [5]:
## This can be hard to read, but printing the 'custom_model_specification' object can let you see what is happening to it. 
custom_model_specification

plugins:
    required:
        component_manager:
            controller:
                base: vivarium.framework.components.ComponentManager
                    source: vivarium_defaults
            builder_interface:
                base: vivarium.framework.components.ComponentInterface
                    source: vivarium_defaults
        clock:
            controller:
                base: vivarium.framework.time.DateTimeClock
                    source: vivarium_defaults
            builder_interface:
                base: vivarium.framework.time.TimeInterface
                    source: vivarium_defaults
        component_configuration_parser:
            controller:
                model_override: vivarium_nih_us_cvd.plugins.CausesConfigurationParser
                    source: /ihme/homes/lutzes/.conda/envs/cvd_eng4/lib/python3.9/site-packages/vivarium_nih_us_cvd/model_specifications/nih_us_cvd.yaml
                base: vivarium.framework.components.ComponentConfigurationPars

# Step 3

In [6]:
sim = InteractiveContext(custom_model_specification)

2023-12-22 10:08:19.316 | INFO     | simulation_1-artifact_manager:67 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/artifacts/51-locations/v4-20231218/alabama.hdf.
2023-12-22 10:08:19.318 | INFO     | simulation_1-artifact_manager:68 - Artifact base filter terms are ['draw == 0'].
2023-12-22 10:08:19.319 | INFO     | simulation_1-artifact_manager:69 - Artifact additional filter terms are None.
2023-12-22 10:09:58.695 | WARNING  | simulation_1-values_manager:279 - Unsourced pipelines: ['disability_weight']
2023-12-22 10:09:58.704 | WARNING  | simulation_1-resource_manager:256 - Resource missing_value_source.disability_weight is not provided by any component but is needed to compute (value.disability_weight).


# Step 4

In [7]:
pop0 = sim.get_population()
pop0.head()

,tracked,location,alive,exit_time,sex,entrance_time,age,years_of_life_lost,cause_of_death,high_systolic_blood_pressure_propensity,...,ldlc_medication,lifestyle,ldlc_multiplier,ldlc_medication_adherence,sbp_multiplier,sbp_medication_adherence,polypill,visit_type,scheduled_date,last_fpg_test_date
0,True,Alabama,alive,NaT,Female,2020-12-04,47.388882,0.0,not_dead,0.843299,...,medium_intensity,NaT,1.3620,cat3,1.051,cat3,cat2,none,NaT,NaT
1,True,Alabama,alive,NaT,Male,2020-12-04,41.564194,0.0,not_dead,0.788239,...,high_intensity,NaT,1.5125,cat3,1.051,cat3,cat2,none,NaT,NaT
2,True,Alabama,alive,NaT,Female,2020-12-04,53.155826,0.0,not_dead,0.861751,...,medium_intensity,NaT,1.3620,cat3,1.120,cat3,cat2,none,NaT,NaT
3,True,Alabama,alive,NaT,Male,2020-12-04,30.457692,0.0,not_dead,0.769485,...,no_treatment,NaT,1.0000,cat1,1.120,cat3,cat2,none,NaT,NaT
4,True,Alabama,alive,NaT,Female,2020-12-04,19.554806,0.0,not_dead,0.820149,...,no_treatment,NaT,1.0000,cat3,1.000,cat3,cat2,none,NaT,NaT


In [8]:
pop0.columns

Index(['tracked', 'location', 'alive', 'exit_time', 'sex', 'entrance_time',
       'age', 'years_of_life_lost', 'cause_of_death',
       'high_systolic_blood_pressure_propensity',
       'high_ldl_cholesterol_propensity',
       'high_fasting_plasma_glucose_propensity',
       'high_body_mass_index_in_adults_propensity', 'ischemic_stroke',
       'ischemic_heart_disease_and_heart_failure',
       'ensemble_propensity_risk_factor.high_ldl_cholesterol',
       'ensemble_propensity_risk_factor.high_systolic_blood_pressure',
       'ensemble_propensity_risk_factor.high_body_mass_index_in_adults',
       'ensemble_propensity_risk_factor.high_fasting_plasma_glucose',
       'sbp_medication_adherence_propensity',
       'ldlc_medication_adherence_propensity', 'outreach_propensity',
       'polypill_propensity', 'lifestyle_propensity',
       'susceptible_to_ischemic_stroke_event_count',
       'susceptible_to_ischemic_stroke_event_time',
       'acute_ischemic_stroke_event_time', 'acute_ische

In [9]:
sim.list_values()

['metrics',
 'cause_specific_mortality_rate',
 'acute_ischemic_stroke.incidence_rate',
 'acute_ischemic_stroke.incidence_rate.paf',
 'acute_ischemic_stroke.dwell_time',
 'acute_ischemic_stroke.disability_weight',
 'disability_weight',
 'acute_ischemic_stroke.excess_mortality_rate',
 'acute_ischemic_stroke.excess_mortality_rate.paf',
 'mortality_rate',
 'chronic_ischemic_stroke.dwell_time',
 'chronic_ischemic_stroke.disability_weight',
 'chronic_ischemic_stroke.excess_mortality_rate',
 'chronic_ischemic_stroke.excess_mortality_rate.paf',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate.paf',
 'acute_myocardial_infarction.incidence_rate',
 'heart_failure_from_ischemic_heart_disease.incidence_rate',
 'heart_failure_residual.incidence_rate',
 'acute_myocardial_infarction.incidence_rate.paf',
 'heart_failure_from_ischemic_heart_disease.incidence_rate.paf',
 'heart_failure_residual.incidence_rate.paf',
 's

In [10]:
data1 = pd.concat([pop0,
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                  ], axis=1)
data1.head()

,tracked,location,alive,exit_time,sex,entrance_time,age,years_of_life_lost,cause_of_death,high_systolic_blood_pressure_propensity,...,ldlc_multiplier,ldlc_medication_adherence,sbp_multiplier,sbp_medication_adherence,polypill,visit_type,scheduled_date,last_fpg_test_date,high_ldl,high_sbp
0,True,Alabama,alive,NaT,Female,2020-12-04,47.388882,0.0,not_dead,0.843299,...,1.3620,cat3,1.051,cat3,cat2,none,NaT,NaT,1.872148,139.628132
1,True,Alabama,alive,NaT,Male,2020-12-04,41.564194,0.0,not_dead,0.788239,...,1.5125,cat3,1.051,cat3,cat2,none,NaT,NaT,1.756671,138.432993
2,True,Alabama,alive,NaT,Female,2020-12-04,53.155826,0.0,not_dead,0.861751,...,1.3620,cat3,1.120,cat3,cat2,none,NaT,NaT,1.854479,142.957697
3,True,Alabama,alive,NaT,Male,2020-12-04,30.457692,0.0,not_dead,0.769485,...,1.0000,cat1,1.120,cat3,cat2,none,NaT,NaT,2.090760,131.928065
4,True,Alabama,alive,NaT,Female,2020-12-04,19.554806,0.0,not_dead,0.820149,...,1.0000,cat3,1.000,cat3,cat2,none,NaT,NaT,0.000000,50.000000


# Step 5

In [11]:
sim_data = pd.DataFrame()
for step in list(range(0,2)):
    sim.step()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_ldl_cholesterol.exposure')(pop0.index).rename('high_ldl'),
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp')],
                       axis=1)
    data_t['step'] = step
    sim_data = pd.concat([sim_data, data_t])

In [12]:
sim_data

,tracked,location,alive,exit_time,sex,entrance_time,age,years_of_life_lost,cause_of_death,high_systolic_blood_pressure_propensity,...,ldlc_medication_adherence,sbp_multiplier,sbp_medication_adherence,polypill,visit_type,scheduled_date,last_fpg_test_date,high_ldl,high_sbp,step
0,True,Alabama,alive,NaT,Female,2020-12-04,47.465542,0.0,not_dead,0.843299,...,cat3,1.051,cat3,cat2,background,2021-07-20 20:01:53.160643200,NaT,1.872148,139.628132,0
1,True,Alabama,alive,NaT,Male,2020-12-04,41.640854,0.0,not_dead,0.788239,...,cat3,1.051,cat3,cat2,none,NaT,NaT,1.756671,138.432993,0
2,True,Alabama,alive,NaT,Female,2020-12-04,53.232486,0.0,not_dead,0.861751,...,cat3,1.120,cat3,cat2,none,NaT,NaT,1.854479,142.957697,0
3,True,Alabama,alive,NaT,Male,2020-12-04,30.534352,0.0,not_dead,0.769485,...,cat1,1.120,cat3,cat2,none,NaT,NaT,2.090760,131.928065,0
4,True,Alabama,alive,NaT,Female,2020-12-04,19.631466,0.0,not_dead,0.820149,...,cat3,1.000,cat3,cat2,none,NaT,NaT,0.000000,50.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,True,Alabama,alive,NaT,Male,2020-12-04,77.789667,0.0,not_dead,0.285040,...,cat3,1.051,cat3,cat2,background,2021-06-03 14:12:54.867398400,2020-05-25 20:53:03.533165360,1.952063,121.550265,1
49996,True,Alabama,alive,NaT,Male,2020-12-04,26.542576,0.0,not_dead,0.759018,...,cat2,1.000,cat3,cat2,none,NaT,NaT,2.938039,121.255707,1
49997,True,Alabama,alive,NaT,Female,2020-12-04,69.185895,0.0,not_dead,0.715126,...,cat2,1.120,cat3,cat2,none,NaT,NaT,4.408911,146.522565,1
49998,True,Alabama,alive,NaT,Female,2020-12-04,15.338175,0.0,not_dead,0.607295,...,cat3,1.000,cat3,cat2,none,NaT,NaT,0.000000,50.000000,1
